In [1]:
import sys
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import cv2
import matplotlib.pyplot as plt
from glob import glob
import os 
from os.path import join

from tensorflow.keras.layers import Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.resnet import preprocess_input

In [2]:
def get_images_and_labels(directory):
    os.chdir(directory)
    
    images = []
    images_names = []
    class_images = []

    images_names = glob('*.bmp')

    for img in images_names:
        #separação do label da classe
        label = img.split('0')[0]
        label = label.split('1')[0]
        class_images.append(label)
        
        #abertura da imagem
        pathfile = join(directory, img)
        temporary = cv2.imread(pathfile, cv2.IMREAD_UNCHANGED)
        images.append(temporary)

    
    return images, images_names, class_images

In [3]:
def segmentation(list_image):
    #list_images está em BGR

    bgr_list = []
    for img in list_image:
        simpson = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        simpson_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        
        light_yellow = (0, 70, 110)
        dark_yellow = (130, 255, 255)
        mask = cv2.inRange(simpson_hsv, light_yellow, dark_yellow)

        result_rbg = cv2.bitwise_and(simpson, simpson, mask=mask)
        
        result_bgr = cv2.cvtColor(result_rbg, cv2.COLOR_RGB2BGR)

        bgr_list.append(result_bgr)

    return bgr_list

In [4]:
def generate_hist(list_image, dimensions, equalize=False):
    histogram_list = []

    for img in list_image:
        img = cv2.resize(img, dimensions, interpolation=cv2.INTER_AREA)

        if (equalize == True):
            # convert from RGB color-space to YCrCb
            ycrcb_img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

            # equalize the histogram of the Y channel
            ycrcb_img[:, :, 0] = cv2.equalizeHist(ycrcb_img[:, :, 0])

            # convert back to RGB color-space from YCrCb
            img = cv2.cvtColor(ycrcb_img, cv2.COLOR_YCrCb2BGR)
            

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        histogram = cv2.calcHist([img], [0,1,2], None, [8,8,8], [0, 256, 0, 256, 0, 256])
        cv2.normalize(histogram, histogram, 0, 255, cv2.NORM_MINMAX)
        histogram = histogram.flatten()

        histogram_list.append(histogram)
        
    return histogram_list

In [5]:
def calcHuMoments(img):
	img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
	ret,th = cv2.threshold(img,128,255,cv2.THRESH_BINARY_INV)
	moment = cv2.moments(th)
	huMoment = cv2.HuMoments(moment)

	return map(lambda hu: -1 * np.sign(hu) * np.log10(np.abs(hu)), huMoment)

In [6]:
def generate_hu_moments(list_image):
    hu_moments_list = []

    for img in list_image:
        huMoment = calcHuMoments(img)
        huMoment = list(huMoment)
        huMoment = np.concatenate(huMoment, axis=0)
        temporary = np.array(huMoment)
                
        hu_moments_list.append(temporary[:3])
        
    return hu_moments_list

In [ ]:
def generate_vec_from_cnn(name, base_dir):
    ## PARÂMETROS
    img_rows, img_cols = 224, 224
    input_shape = (img_rows, img_cols, 3)

    #Batch size
    batch_size = 32

    nome_base = 'simpsons'

    path_train = join(base_dir, nome_base, 'train')
    path_test = join(base_dir, nome_base, 'test')


    #Carregamento da Base de Dados
    train_dataset = image_dataset_from_directory(
        path_train,
        image_size=(img_rows, img_cols),
        color_mode="rgb",
        batch_size=batch_size,
        shuffle=False)
    test_dataset = image_dataset_from_directory(
        path_test,
        image_size=(img_rows, img_cols),
        color_mode="rgb",
        batch_size=batch_size,
        shuffle=False)
    
    #OTIMIZAÇÃO
    AUTOTUNE = tf.data.AUTOTUNE
    train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
    test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)
    


    if (name == 'vgg'):
        vgg16 = VGG16()

        vgg16_top_false =  VGG16(include_top=False, input_shape=(224,224,3))

        cnn = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
        inputs = keras.Input(shape=input_shape)
        x = preprocess_input(inputs)
        x = cnn(x)
        output = Flatten()(x)
        model = Model(inputs, output)
    
    
    elif (name == 'resnet50'):
        resnet50 = ResNet50()

        resnet50_top_false =  ResNet50(include_top=False, input_shape=(224,224,3))

        cnn = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
        inputs = keras.Input(shape=input_shape)
        x = preprocess_input(inputs)
        x = cnn(x)
        output = GlobalAveragePooling2D()(x)
        model = Model(inputs, output)
    

    X_train = model.predict(train_dataset)
    X_test = model.predict(test_dataset)

    # y_train = np.concatenate([y for x, y in train_dataset], axis=0)
    # y_test = np.concatenate([y for x, y in test_dataset], axis=0)

    return X_train, X_test

In [7]:
def classification(t_cmp, v_cmp, t_images, v_images, t_classes, v_classes, method):
    classification_count, final_count = 0, 0

    for v_index, validation in enumerate(v_cmp):
        temp_list = []

        for _, compare in enumerate(t_cmp):
            #cálculo da distância euclidiana entre as duas entradas
            distance = np.linalg.norm(validation-compare)
            temp_list.append(distance)
        
        min_value = min(temp_list)
        min_index = temp_list.index(min_value)


        if (v_classes[v_index] == t_classes[min_index]):
            classification_count += 1
            classification_result = "(ACERTO)"
        else:
            classification_result = "(ERRO)"


        final_count += 1
        print(f"{v_images[v_index]} - {t_images[min_index]} -- {classification_result}")

    accuracy = classification_count / final_count
    print( "\n")
    print(f"-->Nome do Método = {method}")
    print( "-->Classificações CORRETAS = {}".format(classification_count) )
    print( "-->Total de IMAGENS = {}".format(final_count) )
    print( "-->Acurácia = {0:1.3f}".format(accuracy) )

In [ ]:
if __name__ == "__main__":
    #DEFINIÇÕES INICIAIS
    dimensions = (450,500)
    base_dir = os.getcwd()
    print(base_dir)

    #Leitura do caminho do treino
    train_path = sys.argv[1]
    print(train_path)

    #Leitura do caminho da validação
    valid_path = sys.argv[2]
    print(valid_path)

    #Leitura das imagens
    os.chdir(base_dir)
    train_images, train_names, train_classes = get_images_and_labels(train_path)
    valid_images, valid_names, valid_classes = get_images_and_labels(valid_path)
    os.chdir(base_dir)

    #Segmentação das imagens
    train_images_seg = segmentation(train_images)
    valid_images_seg = segmentation(valid_images)

    if (len(sys.argv) == 4):
        if sys.argv[3] == "m02": ##histogramas sem equalização
            print("--------- METODOLOGIA BASEADA EM COMPARAÇÃO DE HISTOGRAMAS ---------")
            train_histogram = generate_hist(train_images, dimensions, equalize=False)
            valid_histogram = generate_hist(valid_images, dimensions, equalize=False)
            classification(train_histogram, valid_histogram, train_names, valid_names, train_classes, valid_classes, "Classificação de Histogramas (sem equalização)")

        elif sys.argv[3] == "m03": ##histogramas sem equalização e com segmentação de imagem
            print("--------- METODOLOGIA BASEADA EM COMPARAÇÃO DE HISTOGRAMAS ---------")
            train_histogram = generate_hist(train_images_seg, dimensions, equalize=False)
            valid_histogram = generate_hist(valid_images_seg, dimensions, equalize=False)
            classification(train_histogram, valid_histogram, train_names, valid_names, train_classes, valid_classes, "Classificação de Histogramas (sem equalização e com segmentação de imagem)")

        elif sys.argv[3] == "m04": ##momentos de hu sem segmentação
            print("--------- METODOLOGIA BASEADA EM COMPARAÇÃO DE MOMENTOS DE HU ---------")
            train_moments = generate_hu_moments(train_images)
            valid_moments = generate_hu_moments(valid_images)
            classification(train_moments, valid_moments, train_names, valid_names, train_classes, valid_classes, "Classificação de Momentos de HU (sem segmentação)")

        elif sys.argv[3] == "m05": ##momentos de hu com segmentação
            print("--------- METODOLOGIA BASEADA EM COMPARAÇÃO DE MOMENTOS DE HU ---------")
            train_moments = generate_hu_moments(train_images_seg)
            valid_moments = generate_hu_moments(valid_images_seg)
            classification(train_moments, valid_moments, train_names, valid_names, train_classes, valid_classes, "Classificação de Momentos de HU (com segmentação)")

        elif sys.argv[3] == "m06": ##vgg como extrator de característica
            print("--------- METODOLOGIA BASEADA EM EXTRAÇÃO COM VGG16 ---------")
            train_vectors, valid_vectors = generate_vec_from_cnn('vgg', base_dir)
            classification(train_vectors, valid_vectors, train_names, valid_names, train_classes, valid_classes, "Classificação utilizando VGG16 para extração de CARACTERÍSTICAS")

        elif sys.argv[3] == "m07": ##resnet5 como extrator de característica
            print("--------- METODOLOGIA BASEADA EM EXTRAÇÃO COM ResNet50 ---------")
            train_vectors, valid_vectors = generate_vec_from_cnn('resnet50', base_dir)
            classification(train_vectors, valid_vectors, train_names, valid_names, train_classes, valid_classes, "Classificação utilizando ResNet50 para extração de CARACTERÍSTICAS")
    
    else: ##histogramas com equalização (melhor resultado)
        print("--------- METODOLOGIA BASEADA EM COMPARAÇÃO DE HISTOGRAMAS ---------")
        train_histogram = generate_hist(train_images, dimensions, equalize=True)
        valid_histogram = generate_hist(valid_images, dimensions, equalize=True)
        classification(train_histogram, valid_histogram, train_names, valid_names, train_classes, valid_classes, "Classificação de Histogramas (com equalização)")
